# Practicle implementation of RAG & GraphRAG (microsoft)




# Installing libraries

In [14]:
! pip install tantivy
! pip install -U langchain-openai langchain-community
! pip install lancedb
! pip install graphrag==0.1.1
! pip install langchainhub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 14.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of dask-expr to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 361.1/361.1 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 50.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 63.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 70.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.5/25.5 MB 60.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 562.2/562.2 kB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 78.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 63.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━

#Dataset Download

In [15]:
# Create the dataset directory
!mkdir -p rag_exim/input

# Download the dataset into the input directory
!wget -P rag_exim/input https://github.com/akashAD98/dummy_data/raw/main/HTE_gst_scheme.txt

--2024-11-24 06:59:28--  https://github.com/akashAD98/dummy_data/raw/main/HTE_gst_scheme.txt
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/akashAD98/dummy_data/main/HTE_gst_scheme.txt [following]
--2024-11-24 06:59:29--  https://raw.githubusercontent.com/akashAD98/dummy_data/main/HTE_gst_scheme.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 216384 (211K) [text/plain]
Saving to: ‘rag_exim/input/HTE_gst_scheme.txt.1’

HTE_gst_scheme.txt. 100%[===================>] 211.31K  1.29MB/s    in 0.2s    

2024-11-24 06:59:29 (1.29 MB/s) - ‘rag_exim/input/HTE_gst_scheme.txt.1’ saved [216384/216384

In [ ]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = "sk-proj-..."
os.environ["LANGCHAIN_TRACING_V2"] = "False"

In [4]:
import bs4
from langchain import hub
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import LanceDB
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from lancedb.rerankers import LinearCombinationReranker


loader = TextLoader("/content/rag_exim/input/HTE_gst_scheme.txt")
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1100, chunk_overlap=100)
splits = text_splitter.split_documents(docs)
reranker = LinearCombinationReranker(weight=0.3)
vectorstore = LanceDB.from_documents(
    documents=splits, embedding=OpenAIEmbeddings(), reranker=reranker
)

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

query = "tell me how to do export?"

rag_chain.invoke(query)

/usr/local/lib/python3.10/dist-packages/langsmith/client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


"Exporting involves deciding on the mode of shipping, working with a freight forwarder and customs clearance agent, and considering container shipping methods. It is important to procure goods as per buyer's requirements and maintain strict quality control for export goods. Exporting can lead to increasing profit margins, reducing production costs per unit, and improving liquidity through favorable payment terms."

#GraphRAG

In [ ]:
# please pass opemai api key
import os

os.environ["GRAPHRAG_API_KEY"] = "sk-proj-..."

#### Step 1. *This* command sets up the initial structure for indexing your dataset.


In [16]:
!python3 -m graphrag.index --init --root /content/rag_exim

2024-11-24 06:59:56.100411: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-24 06:59:56.140990: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-24 06:59:56.153858: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-24 06:59:58.188647: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Initializing project at /content/rag_exim
⠋ GraphRAG Indexer 

#### Step 2. Indexing the **dataset**



In [17]:
!python3 -m graphrag.index --root  /content/rag_exim

Streaming output truncated to the last 5000 lines.
└── create_summarized_entities
⠧ GraphRAG Indexer 
├── Loading Input (text) - 1 files loaded (1 filtered) ━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00 0:00:00
├── create_base_text_units
├── create_base_extracted_entities
└── create_summarized_entities
⠋ GraphRAG Indexer 
├── Loading Input (text) - 1 files loaded (1 filtered) ━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00 0:00:00
├── create_base_text_units
├── create_base_extracted_entities
└── create_summarized_entities
⠹ GraphRAG Indexer 
├── Loading Input (text) - 1 files loaded (1 filtered) ━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00 0:00:00
├── create_base_text_units
├── create_base_extracted_entities
└── create_summarized_entities
⠸ GraphRAG Indexer 
├── Loading Input (text) - 1 files loaded (1 filtered) ━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00 0:00:00
├── create_base_text_units
├── create_base_extracted_entities
└── create_summarized_entities
⠼ GraphRAG Indexer 
├── Loading Input (text) - 1 files loaded 

#### step 3. Execute the query


In [18]:
!python3 -m graphrag.query --root /content/rag_exim --method global "tell me how to do export?"

2024-11-24 07:07:02.592036: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-24 07:07:02.620399: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-24 07:07:02.630765: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-24 07:07:04.281736: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


INFO: Reading settings from /content/rag_exim/settings.yaml
creating llm client with {'api_key': 'REDACTED,len=56', 'type': "openai_chat", 'model': 'gpt-4-turbo-preview', 'max_tokens': 4000, 'request_timeout': 180.0, 'api_base': None, 'api

In [19]:
!python3 -m graphrag.query --root /content/rag_exim --method local "Tell me how to do export ?"

2024-11-24 07:07:55.064127: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-24 07:07:55.088098: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-24 07:07:55.096078: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-24 07:07:56.557671: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


INFO: Reading settings from /content/rag_exim/settings.yaml
[2024-11-24T07:08:08Z WARN  lance::dataset] No existing dataset at /content/lancedb/description_embedding.lance, it will be created
creating llm client with {'api_key': 'REDACTED,

#### Query local search

In [20]:
!python3 -m graphrag.query --root /content/rag_exim --method local "what are the different Payment Methods in International Trade"

2024-11-24 07:08:34.992713: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-24 07:08:35.017903: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-24 07:08:35.024885: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-24 07:08:36.477401: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


INFO: Reading settings from /content/rag_exim/settings.yaml
creating llm client with {'api_key': 'REDACTED,len=56', 'type': "openai_chat", 'model': 'gpt-4-turbo-preview', 'max_tokens': 4000, 'request_timeout': 180.0, 'api_base': None, 'api

### Below is python code you can use it for your chatbot application

In [ ]:
import os
import pandas as pd
import tiktoken
import asyncio
from graphrag.query.indexer_adapters import read_indexer_entities, read_indexer_reports
from graphrag.query.llm.oai.chat_openai import ChatOpenAI
from graphrag.query.llm.oai.typing import OpenaiApiType
from graphrag.query.structured_search.global_search.community_context import (
    GlobalCommunityContext,
)
from graphrag.query.structured_search.global_search.search import GlobalSearch

# ## Global Search example
api_key = os.environ["GRAPHRAG_API_KEY"] = "sk-proj-...."
llm_model = "gpt-3.5-turbo"

llm = ChatOpenAI(
    api_key=api_key,
    model=llm_model,
    api_type=OpenaiApiType.OpenAI,  # OpenaiApiType.OpenAI or OpenaiApiType.AzureOpenAI
    max_retries=20,
)


print("llm loaded okay")

token_encoder = tiktoken.get_encoding("cl100k_base")

INPUT_DIR = (
    "/content/rag_exim/output/"  # path of output folder which has all parquete files
)

for f in os.listdir(INPUT_DIR):
    for artifact in os.listdir(INPUT_DIR + f):
        if artifact == "artifacts":
            artifacts_directory = INPUT_DIR + f + "/artifacts/"
            break

INPUT_DIR = artifacts_directory

COMMUNITY_REPORT_TABLE = "create_final_community_reports"
ENTITY_TABLE = "create_final_nodes"
ENTITY_EMBEDDING_TABLE = "create_final_entities"

# community level in the Leiden community hierarchy from which we will load the community reports
# higher value means we use reports from more fine-grained communities (at the cost of higher computation cost)
COMMUNITY_LEVEL = 2

# %%
entity_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_TABLE}.parquet")
report_df = pd.read_parquet(f"{INPUT_DIR}/{COMMUNITY_REPORT_TABLE}.parquet")
entity_embedding_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_EMBEDDING_TABLE}.parquet")

reports = read_indexer_reports(report_df, entity_df, COMMUNITY_LEVEL)
entities = read_indexer_entities(entity_df, entity_embedding_df, COMMUNITY_LEVEL)
print(f"Report records: {len(report_df)}")
report_df.head()

# #### Build global context based on community reports
context_builder = GlobalCommunityContext(
    community_reports=reports,
    entities=entities,  # default to None if you don't want to use community weights for ranking
    communities=token_encoder,
)

# #### Perform global search

context_builder_params = {
    "use_community_summary": False,  # False means using full community reports. True means using community short summaries.
    "shuffle_data": True,
    "include_community_rank": True,
    "min_community_rank": 0,
    "community_rank_name": "rank",
    "include_community_weight": True,
    "community_weight_name": "occurrence weight",
    "normalize_community_weight": True,
    "max_tokens": 3_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 5000)
    "context_name": "Reports",
}

map_llm_params = {
    "max_tokens": 1000,
    "temperature": 0.0,
    "response_format": {"type": "json_object"},
}

reduce_llm_params = {
    "max_tokens": 2000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 1000-1500)
    "temperature": 0.0,
}

search_engine = GlobalSearch(
    llm=llm,
    context_builder=context_builder,
    token_encoder=token_encoder,
    max_data_tokens=12_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 5000)
    map_llm_params=map_llm_params,
    reduce_llm_params=reduce_llm_params,
    allow_general_knowledge=False,  # set this to True will add instruction to encourage the LLM to incorporate general knowledge in the response, which may increase hallucinations, but could be useful in some use cases.
    json_mode=True,  # set this to False if your LLM model does not support JSON mode.
    context_builder_params=context_builder_params,
    concurrent_coroutines=10,
    response_type="multiple-page report",  # free form text describing the response type and format, can be anything, e.g. prioritized list, single paragraph, multiple paragraphs, multiple-page report
)

query = "what are the different Payment Methods in International Trade"
result = await search_engine.asearch(query)
print(result.response)
print("____________________________________")
print(f"LLM calls: {result.llm_calls}. LLM tokens: {result.prompt_tokens}")

llm loaded okay
Report records: 110
# Summary of Payment Methods in International Trade

In international trade, various payment methods play a crucial role in facilitating transactions between buyers and sellers. Analysts have highlighted the following key payment methods:

## Letters of Credit (LC)
Letters of Credit are a common and widely used payment method in international trade. They involve the buyer's bank guaranteeing payment to the seller upon the presentation of specified documents. This method provides security for both parties by ensuring that the seller receives payment once the conditions of the letter of credit are met [Data: Reports (106, 78, 46, 66, 5, +more)].

## Documentary Collections
Documentary Collections involve the exchange of shipping documents for payment. Banks act as intermediaries to ensure the transfer of goods and payment, providing a compromise between security and flexibility for both the buyer and the seller [Data: Reports (29, 24, 74, 59)].

## Ope

You can clearly see the difference in output quality between the two methods. GraphRAG is able to extract deeper insights from graph nodes, whereas traditional RAG struggles in this area. However, this comes at a higher cost - GraphRAG requires 11 LLM calls, leading to increased token usage.
Both methods have their trade-offs, but there are ways to enhance their performance. For GraphRAG, using a local LLaMA model could reduce costs, while RAG has various optimization techniques available. Check out our repository for more details on improving both approaches